In [ ]:
/************************************************************************
 * Last Run Date: 24 April 2017	            							*
 * Program Name: panders2_hw12_prog.sas									*
 * Program Location: C:/Users/Philip/Schools/TAMU/STAT_657/				*
 *				SASUniversityEdition/myfolders/homework/git_hw/hw_13/sasprogs/	* 
 * Creation Date: 24 April 2017											*
 * Author: Philip Anderson												*
 * Purpose: Meeting objectives for Assignment 13						*
 * Inputs: basketball data         *
 * Output: panders2_hw13_output.pdf															*
 * Modification History: *
 * Execution Instructions: N/A - run as-is												*
*/**********************************************************************;

In [17]:
/*specify the option that will print information about the indexes */
options msglevel=i nonumber nodate;
libname bball '/folders/myfolders/homework/git_hw/hw_04/sasdata' access=readonly;
libname sasdata '/folders/myfolders/homework/git_hw/hw_13/sasdata';


In [16]:
/*Question 1 - recreate the output of p306d02 using SQL */
/*SAS is as follows: */
/* data country;
   keep Start Label FmtName;
   retain FmtName '$Country';
   set orion.Country(rename=(Country=Start
                              Country_Name = Label));
run;
 */
 
proc sql;
create table test as
select 
distinct
'$seedz' as FmtName 
, team as Start
, put(seed_, 2.) as Label
from
bball.ncaam04
;

insert into test
    values ('$seedz', 'other', 'NA')
;
quit;

proc print data=test; run; 


Obs,FmtName,Start,Label
1,$seedz,Boston College,6
2,$seedz,Connecticut,2
3,$seedz,Duke,1
4,$seedz,Georgia Tech,3
5,$seedz,Gonzaga,2
6,$seedz,Kansas,4
7,$seedz,Kentucky,1
8,$seedz,Maryland,4
9,$seedz,Mississippi State,2
10,$seedz,North Carolina,6


In [18]:
proc format library=sasdata.myfmts cntlin=test fmtlib;
    select $seedz;
run;


In [50]:
/* Question 2 - create a picture format that can be applied to the PPG column 
allow space for PPG to be displayed as two digits along with a single decimal place (00.0)*/

/* proc print data=bball.ncaam04 (obs=10); run; */

proc format;
    picture ppg_frmtr
    15 - high = '(00.0)' (prefix='High(')
    7.7 -< 15 = '(00.0)' (prefix='Medium(')
    low -< 7.7 = '(00.0)' (prefix='Low(')
    ;
run;



In [43]:
/*Question 3 - use a SAS procedure to place a copy of the ncaam06 dataset in the WORK library */
proc datasets noprint;
copy in=bball out=work;
select ncaam06;
quit;



In [60]:
proc sql;
create table ncaam06_update as
select 
ncaam06.*
, ppg as ppg_rating 'PPG Rating' format ppg_frmtr.
, (case
    when School = 'Indania' then 'Indiana'
    when School = 'Boston Coll' then 'Boston College'
    when School = 'George mason' then 'George Mason'
    when School = 'Oral Robt-16' then 'Oral Roberts'
    when School = 'Wisc. Milwaukee' then 'Wisconsin Milwaukee'
    else School end) as School
from
ncaam06
;
quit;

In [65]:

/*Question 5 - create a composite index for ncaam06 using player, school, and region-  in that order */
data ncaam06_update 
    (index=(bball_idx=(player school region )));
set ncaam06_update;
run;

proc contents data=ncaam06_update; run;

In [68]:
/*Question 6 - create a print procedure that will print the columns shown in the output */

title '6a. IDXWHERE on Player';
proc print data=ncaam06_update (keep=player school region seed ppg seed);
where player in ('Steve Burtt', 'Jared Dudley', 'Stanley Burrell');
run;

title '6b. IDXWHERE on School';

title '6c. IDXNAME on School';

title '6d. IDXWHERE on Player or School';

title '6e. IDXWHERE on Player and School';



Obs,Seed,School,Region,Player,PPG
8,4,Boston Coll,MIN,Jared Dudley,16.5
193,14,Xavier,OAK,Stanley Burrell,14.1
57,13,Iona,ATL,Steve Burtt,25.2
